<a href="https://colab.research.google.com/github/zw2497/Twitter_Stream_Processing/blob/master/Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
consumer_key = "voHiOtlHtgaHC92ek5NYTABAZ"
consumer_secret = "qHra6hGcG7wWEEgTP0mrysqHquaLaloF2tkHcNJfM1tcskImgj"
access_token = "1114974821434241024-BZrPQPmONxVrb6NUVnVa6QQwBpy1An"
access_token_secret = "lRqIvcWURI351ixfT5QMogFgHNajKAwG2cTVPKeclGNXP"

In [0]:
import tweepy

class MyStreamListener(tweepy.StreamListener):

   def on_status(self, status):
       print(status.text)

   def on_error(self, status_code):
       if status_code == 420:
           #returning False in on_data disconnects the stream
           return False

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)

In [0]:
myStream.filter(languages=['english'])

In [2]:
!pip install pycurl
!pip install oauth2

In [0]:
import time
import pycurl
import urllib
import json
import oauth2 as oauth

In [0]:
API_ENDPOINT_URL = 'https://stream.twitter.com/1.1/statuses/filter.json'
USER_AGENT = 'TwitterStream 1.0' # This can be anything really

In [5]:
# You need to replace these with your own values
OAUTH_KEYS = {'consumer_key': consumer_key,
              'consumer_secret': consumer_secret,
              'access_token_key': access_token,
              'access_token_secret': access_token_secret}
print(OAUTH_KEYS)

{'consumer_secret': 'qHra6hGcG7wWEEgTP0mrysqHquaLaloF2tkHcNJfM1tcskImgj', 'access_token_key': '1114974821434241024-BZrPQPmONxVrb6NUVnVa6QQwBpy1An', 'consumer_key': 'voHiOtlHtgaHC92ek5NYTABAZ', 'access_token_secret': 'lRqIvcWURI351ixfT5QMogFgHNajKAwG2cTVPKeclGNXP'}


In [0]:
POST_PARAMS = {'include_entities': 0,
               'stall_warning': 'true',
               'track': 'iphone,ipad,ipod'}

In [0]:
class TwitterStream:
    def __init__(self, timeout=False):
        self.oauth_token = oauth.Token(key=OAUTH_KEYS['access_token_key'], secret=OAUTH_KEYS['access_token_secret'])
        self.oauth_consumer = oauth.Consumer(key=OAUTH_KEYS['consumer_key'], secret=OAUTH_KEYS['consumer_secret'])
        self.conn = None
        self.buffer = ''
        self.timeout = timeout
        self.setup_connection()

    def setup_connection(self):
        """ Create persistant HTTP connection to Streaming API endpoint using cURL.
        """
        if self.conn:
            self.conn.close()
            self.buffer = ''
        self.conn = pycurl.Curl()
        # Restart connection if less than 1 byte/s is received during "timeout" seconds
        if isinstance(self.timeout, int):
            self.conn.setopt(pycurl.LOW_SPEED_LIMIT, 1)
            self.conn.setopt(pycurl.LOW_SPEED_TIME, self.timeout)
        self.conn.setopt(pycurl.URL, API_ENDPOINT_URL)
        self.conn.setopt(pycurl.USERAGENT, USER_AGENT)
        # Using gzip is optional but saves us bandwidth.
        self.conn.setopt(pycurl.ENCODING, 'deflate, gzip')
        self.conn.setopt(pycurl.POST, 1)
        self.conn.setopt(pycurl.POSTFIELDS, urllib.urlencode(POST_PARAMS))
        self.conn.setopt(pycurl.HTTPHEADER, ['Host: stream.twitter.com',
                                             'Authorization: %s' % self.get_oauth_header()])
        # self.handle_tweet is the method that are called when new tweets arrive
        self.conn.setopt(pycurl.WRITEFUNCTION, self.handle_tweet)

    def get_oauth_header(self):
        """ Create and return OAuth header.
        """
        params = {'oauth_version': '1.0',
                  'oauth_nonce': oauth.generate_nonce(),
                  'oauth_timestamp': str(int(time.time()))}
        req = oauth.Request(method='POST', parameters=params, url='%s?%s' % (API_ENDPOINT_URL,
                                                                             urllib.urlencode(POST_PARAMS)))
        req.sign_request(oauth.SignatureMethod_HMAC_SHA1(), self.oauth_consumer, self.oauth_token)
        
       
        return str(req.to_header()['Authorization']).encode('utf-8')

    def start(self):
        """ Start listening to Streaming endpoint.
        Handle exceptions according to Twitter's recommendations.
        """
        backoff_network_error = 0.25
        backoff_http_error = 5
        backoff_rate_limit = 60
        while True:
            self.setup_connection()
            try:
                self.conn.perform()
            except:
                # Network error, use linear back off up to 16 seconds
                print('Network error: %s' % self.conn.errstr())
                print('Waiting %s seconds before trying again' % backoff_network_error)
                time.sleep(backoff_network_error)
                backoff_network_error = min(backoff_network_error + 1, 16)
                continue
            # HTTP Error
            sc = self.conn.getinfo(pycurl.HTTP_CODE)
            if sc == 420:
                # Rate limit, use exponential back off starting with 1 minute and double each attempt
                print('Rate limit, waiting %s seconds' % backoff_rate_limit)
                time.sleep(backoff_rate_limit)
                backoff_rate_limit *= 2
            else:
                # HTTP error, use exponential back off up to 320 seconds
                print('HTTP error %s, %s' % (sc, self.conn.errstr()))
                print('Waiting %s seconds' % backoff_http_error)
                time.sleep(backoff_http_error)
                backoff_http_error = min(backoff_http_error * 2, 320)

    def handle_tweet(self, data):
        """ This method is called when data is received through Streaming endpoint.
        """
        self.buffer += str(data)
        if data.endswith(b'\r\n') and self.buffer.strip():
            # complete message received
            message = json.loads(self.buffer)
            self.buffer = ''
            msg = ''
            if message.get('limit'):
                print('Rate limiting caused us to miss %s tweets' % (message['limit'].get('track')))
            elif message.get('disconnect'):
                raise Exception('Got disconnect: %s' % message['disconnect'].get('reason'))
            elif message.get('warning'):
                print('Got warning: %s' % message['warning'].get('message'))
            else:
                print(message)
                print('Got tweet with text: %s' % message.get('text'))

In [10]:
ts = TwitterStream()
ts.setup_connection()
ts.start()

{u'quote_count': 0, u'contributors': None, u'truncated': False, u'text': u'And by photoshoot I mean Shannon\u2019s iPhone', u'is_quote_status': False, u'in_reply_to_status_id': None, u'reply_count': 0, u'id': 1118662136752787457, u'favorite_count': 0, u'source': u'<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', u'retweeted': False, u'coordinates': None, u'timestamp_ms': u'1555544811796', u'entities': {u'user_mentions': [], u'symbols': [], u'hashtags': [], u'urls': []}, u'in_reply_to_screen_name': None, u'id_str': u'1118662136752787457', u'retweet_count': 0, u'in_reply_to_user_id': None, u'favorited': False, u'user': {u'follow_request_sent': None, u'profile_use_background_image': True, u'default_profile_image': False, u'id': 105042381, u'default_profile': False, u'verified': False, u'profile_image_url_https': u'https://pbs.twimg.com/profile_images/498997197590237184/XCleq7A3_normal.jpeg', u'profile_sidebar_fill_color': u'00AABD', u'profile_text_color

KeyboardInterrupt: ignored

Network error: Failed writing body (0 != 5031)
Waiting 0.25 seconds before trying again
{u'quote_count': 0, u'quoted_status_permalink': {u'url': u'https://t.co/9x4qvOePDZ', u'expanded': u'https://twitter.com/AAronblockaye/status/1116410564803223552', u'display': u'twitter.com/AAronblockaye/\u2026'}, u'contributors': None, u'truncated': False, u'text': u'RT @IamDeno623: Kik, the KDs, and iPhone 5c yea.... but never have i ever rocked the emoji fit \U0001f602\U0001f645\U0001f3fe\u200d\u2642\ufe0f https://t.co/9x4qvOePDZ', u'is_quote_status': True, u'in_reply_to_status_id': None, u'reply_count': 0, u'id': 1118662153618305024, u'favorite_count': 0, u'entities': {u'user_mentions': [{u'id': 721420031091822592, u'indices': [3, 14], u'id_str': u'721420031091822592', u'screen_name': u'IamDeno623', u'name': u'Dean Tate \u26a1\ufe0f'}], u'symbols': [], u'hashtags': [], u'urls': [{u'url': u'https://t.co/9x4qvOePDZ', u'indices': [102, 125], u'expanded_url': u'https://twitter.com/AAronblockaye/statu

KeyboardInterrupt: ignored

Network error: Failed writing body (0 != 8614)
Waiting 1.25 seconds before trying again
{u'quote_count': 0, u'contributors': None, u'truncated': False, u'text': u'@BFTrick I play this game with my kids all the time. They\u2019re always stealing my phone or iPad.', u'is_quote_status': False, u'in_reply_to_status_id': 1118572337828331520, u'reply_count': 0, u'id': 1118662170273882112, u'favorite_count': 0, u'entities': {u'user_mentions': [{u'id': 19366137, u'indices': [0, 8], u'id_str': u'19366137', u'screen_name': u'BFTrick', u'name': u'Patrick Rauland'}], u'symbols': [], u'hashtags': [], u'urls': []}, u'retweeted': False, u'coordinates': None, u'timestamp_ms': u'1555544819788', u'source': u'<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', u'in_reply_to_screen_name': u'BFTrick', u'id_str': u'1118662170273882112', u'display_text_range': [9, 94], u'retweet_count': 0, u'in_reply_to_user_id': 19366137, u'favorited': False, u'user': {u'follow_request_sent'

KeyboardInterrupt: ignored

Network error: Failed writing body (0 != 5164)
Waiting 2.25 seconds before trying again
{u'quote_count': 0, u'contributors': None, u'truncated': False, u'text': u'RT @TankGiveaways: iPhone XR Giveaway \n\n-Just RT \n-Follow us\n-Tag Friends\n\nWinner soon\nGood Luck https://t.co/I9blI3UtDI', u'is_quote_status': False, u'in_reply_to_status_id': None, u'reply_count': 0, u'id': 1118662183276072960, u'favorite_count': 0, u'entities': {u'user_mentions': [{u'id': 1038705222141595654, u'indices': [3, 17], u'id_str': u'1038705222141595654', u'screen_name': u'TankGiveaways', u'name': u'TankGiveaways'}], u'symbols': [], u'hashtags': [], u'urls': [], u'media': [{u'source_user_id': 1038705222141595654, u'source_status_id_str': u'1118632034166034437', u'expanded_url': u'https://twitter.com/TankGiveaways/status/1118632034166034437/photo/1', u'display_url': u'pic.twitter.com/I9blI3UtDI', u'url': u'https://t.co/I9blI3UtDI', u'media_url_https': u'https://pbs.twimg.com/media/D4Yt517W4AAn2Sv.jpg', u'sour

KeyboardInterrupt: ignored

Network error: Failed writing body (0 != 13518)
Waiting 3.25 seconds before trying again


KeyboardInterrupt: ignored